In [1]:
import tensorflow.compat.v2 as tf
import tensorflow_datasets as tfds
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [2]:
dataset_name = 'cats_vs_dogs'
dataset, info = tfds.load(name=dataset_name, 
                          split=tfds.Split.TRAIN,
                          with_info=True)


In [3]:
def normalize_img(x):
  """Normalizes images: `uint8` -> `float32`."""
  x['image'] = tf.cast(x['image'], tf.float32) / 255.
  x['image'] = tf.image.resize(x['image'], [224,224])
  return x

ds_train = dataset.map(
    normalize_img, num_parallel_calls=tf.data.experimental.AUTOTUNE)

In [4]:
import numpy as np
train_images =[]
train_labels = []
for example in ds_train.take(1200):
  train_images.append(np.array(example["image"]).astype(float))
  train_labels.append(np.array(example["label"]).astype(float))


In [5]:
train_images[1]

array([[[0.8355239 , 0.63748467, 0.43846506],
        [0.88309479, 0.66133583, 0.47408092],
        [0.96694458, 0.71880251, 0.54320514],
        ...,
        [0.3750219 , 0.35541406, 0.06954438],
        [0.39081103, 0.39294249, 0.11197479],
        [0.39285934, 0.41263789, 0.13755253]],

       [[0.81665134, 0.62200636, 0.41149992],
        [0.94470412, 0.68478209, 0.49626884],
        [0.88511688, 0.525267  , 0.37242869],
        ...,
        [0.41858587, 0.38510591, 0.12367821],
        [0.39685747, 0.38381913, 0.11529455],
        [0.36990109, 0.37591475, 0.09940039]],

       [[0.84143692, 0.59477419, 0.3862395 ],
        [0.95612311, 0.63944328, 0.45424765],
        [0.84091604, 0.3950499 , 0.2544446 ],
        ...,
        [0.53460479, 0.48207286, 0.25139838],
        [0.62986475, 0.59536064, 0.34855568],
        [0.39670432, 0.38240767, 0.11501443]],

       ...,

       [[0.34345472, 0.34421656, 0.14737627],
        [0.31167316, 0.31224456, 0.11597566],
        [0.38111863, 0

In [6]:
train_labels[1]

array(1.)

In [7]:
train_datagen = ImageDataGenerator(
      rotation_range=40,
      width_shift_range=0.2,
      height_shift_range=0.2,
      shear_range=0.2,
      zoom_range=0.2,
      horizontal_flip=True,
      fill_mode='nearest')

In [8]:
model = tf.keras.models.Sequential([
    tf.keras.layers.Conv2D(32, (3, 3), activation='relu', input_shape=(224, 224, 3)),
    tf.keras.layers.MaxPooling2D(2, 2),
    tf.keras.layers.Dropout(0.2),
    
    tf.keras.layers.Conv2D(64, (3, 3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2, 2),
    tf.keras.layers.Dropout(0.2),

    tf.keras.layers.Conv2D(128, (3, 3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2, 2),
    tf.keras.layers.Dropout(0.2),

    tf.keras.layers.Flatten(),

    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dense(2, activation='softmax')])

# Compile Model. 
model.compile(optimizer = 'adam',
              loss = 'sparse_categorical_crossentropy',
              metrics=['accuracy'])

In [11]:
train_images = np.array(train_images)
train_labels=np.array(train_labels)

In [12]:
train_gen = train_datagen.flow(
    train_images,
    train_labels,
    batch_size=32
)

In [13]:
history = model.fit_generator(
    train_gen,
    epochs=3
)

Instructions for updating:
Please use Model.fit, which supports generators.


Instructions for updating:
Please use Model.fit, which supports generators.


Epoch 1/3
38/38 [==============================] - 118s 3s/step - loss: 1.2903 - accuracy: 0.5033
Epoch 2/3
38/38 [==============================] - 118s 3s/step - loss: 0.6930 - accuracy: 0.5100
Epoch 3/3
38/38 [==============================] - 121s 3s/step - loss: 0.6932 - accuracy: 0.5108


In [14]:
model.save('cats_dogs_checker.h5')

In [15]:
import os
print(str(os.path.getsize('./cats_dogs_checker.h5')/1000000) + 'MB')

134.08696MB
